# Chatbot usando la arquitectura RAG y un modelo LLM

Este notebook carga un modelo de lenguaje desde una carpeta local si se encuentra en Jetson Nano y proporciona una interfaz chatbot para responder cualquier inquietud acerca del Laboratorio Alan Turing

**IMPORTANTE:** Asegúrate de tener la carpeta si es local con los dos modelos en la misma ubicación que este notebook.

### Instalación de Librerías

Ejecuta esta celda si no se han instalado las librerias previamente.

In [1]:
!pip3 install --upgrade pip setuptools wheel
!pip3 install -q torch torchvision transformers ipywidgets

In [2]:
# Esto es en caso que se pueda lanzar en gradio, lo cual para la version python 3.6 causo problemas de compatibilidad
# !pip3 install --upgrade gradio

In [3]:
!pip3 install beautifulsoup4 sentence_transformers faiss-cpu

In [4]:
!find /usr/lib -name "libgomp.so.1"

/usr/lib/x86_64-linux-gnu/libgomp.so.1


In [ ]:
# Para linux (jetson nano)
%env LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1

import sklearn
import requests
import re
import faiss
import numpy as np
import os

from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer
import torch
# -----------------------------------------------------------------------------

# --- CONFIGURACIÓN DE MODELOS Y RAG ---
# sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 o intfloat/multilingual-e5-small
# Para mi caso en jetson nano se descargo manualmente de huggingface los modelos y se pone la ubicacion correspondiente /dli-nano/LLMs/modelo, en maquinas aparte se uso las urls

LLM_MODEL_NAME = "google/flan-t5-small"
EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-small"
# Nombre de los indexados
INDEX_NAME = "lab_ia_epn.faiss"
CHUNKS_NAME = "lab_ia_epn_chunks.npy"
# --------------------------------

env: LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1


In [ ]:
# Conocimiento adquirido desde la pagina web del Laboratorio Alan Turing
knowledge_chunks_manual = [
    "El Laboratorio Alan Turing fue fundado el 17 de julio de 2018. La fecha de creación se estableció mediante resolución del Consejo de la Facultad de Ingeniería de Sistemas de la EPN.",
    "El Laboratorio de Investigación en Inteligencia y Visión Artificial “Alan Turing” fue creado mediante resolución del Consejo de la Facultad de Ingeniería de Sistemas de la Escuela Politécnica Nacional, el 17 de julio de 2018. Nuestro objetivo es desarrollar investigación fundamental y aplicada para desarrollar algoritmos de inteligencia y visión artificial.",
    "El Laboratorio de Investigación en Inteligencia y Visión Artificial “Alan Turing” se dedica al desarrollo de algoritmos de inteligencia y visión artificial. La meta es llevar a cabo investigación tanto fundamental como aplicada en estas áreas.",
    "El Laboratorio Alan Turing (EPN) se creó en julio de 2018 para desarrollar algoritmos de inteligencia artificial y visión artificial. Se fundó por resolución del Consejo de la Facultad de Ingeniería de Sistemas.",
    "La visión del Laboratorio Alan Turing es ser un referente nacional e internacional en la investigación y desarrollo de Inteligencia y Visión Artificial.",
    "Tambien se lo conoce con el nombre completo de Laboratorio de Investigacion en Inteligencia y Vision Artificial 'Alan Turing'",

    "Las redes sociales del Laboratorio Alan Turing son Facebook: https://www.facebook.com/laboratorio.IA.EPN, LinkedIn: https://www.linkedin.com/company/laboratorio-ia-alan-turing, Youtube: https://www.youtube.com/channel/UCWE-OhGIh4rB6bp_MwKWQwQ, Instagram: https://www.instagram.com/labturing.ia.epn?utm_source=qr",
    "El Laboratorio Alan Turing mantiene presencia oficial en cuatro redes sociales clave: Facebook (https://www.facebook.com/laboratorio.IA.EPN), LinkedIn (https://www.linkedin.com/company/laboratorio-ia-alan-turing), YouTube (https://www.youtube.com/channel/UCWE-OhGIh4rB6bp_MwKWQwQ) y Instagram (https://www.instagram.com/labturing.ia.epn?utm_source=qr).",
    "La dirección de correo oficial del laboratorio para consultas generales es laboratorio.ia@epn.edu.ec.. El teléfono de contacto es 2 2976300 ext. 4706. Con direccion Ladrón de Guevara E11-253, Quito – Ecuador Campus Politécnico “José Rubén Orellana” Facultad de Ingeniería de Sistemas en el Cuarto Piso",
    "Para contactar al Laboratorio Alan Turing, puedes usar el correo laboratorio.ia@epn.edu.ec o llamar al número (2) 2976300, extensión 4706. El Laboratorio Alan Turing se ubica en el Campus Politécnico “José Rubén Orellana” de la EPN, específicamente en la Facultad de Ingeniería de Sistemas en el Cuarto Piso.",
    "La ubicación física exacta del Laboratorio Alan Turing es: Ladrón de Guevara E11-253, dentro del Campus Politécnico José Rubén Orellana, específicamente en la Facultad de Ingeniería de Sistemas, Cuarto Piso, Quito – Ecuador.",
    "La dirección de correo oficial del laboratorio para consultas generales es laboratorio.ia@epn.edu.ec. El teléfono de contacto es 2 2976300 ext. 4706.",

    "El Dr. Marco Benalcázar es el director del laboratorio Alan Turing actualmente.",
    "El director actual del Laboratorio Alan Turing es el Dr. Marco E. Benalcázar.",
    "El liderazgo del Laboratorio Alan Turing está a cargo del Dr. Marco E. Benalcázar.",

    "El equipo de investigadores principales del Laboratorio Alan Turing incluye a la Dra. Lorena Barona, el Dr. Leonardo Valdivieso, el Dr. Patricio Cruz, el Dr. Robin Álvarez, Dr. Marco Benalcázar y el Msc. Pablo Marcillo.",
    "El equipo de investigadores principales está compuesto por la Dra. Lorena Barona y los doctores Leonardo Valdivieso, Patricio Cruz, Robin Álvarez, Marco Benalcázar, además del Msc. Pablo Marcillo.",
    "Los colaboradores y estudiantes de doctorado del Laboratorio Alan Turing son: Jonathan A. Zea (MSc.), Mario Moreno (MSc.), Carlos Ayala (MSc.) y Gabriel Macías (MSc.).",
    "El laboratorio Alan Turing cuenta con MSc. Jonathan A. Zea, MSc. Mario Moreno, MSc. Carlos Ayala, y MSc. Gabriel Macías como colaboradores activos, tambien realizan estudios de doctorado.",
    "El Laboratorio Alan Turing cuenta con once pasantes activos en total. Los nombres de los pasantes incluyen a: Pablo Maldonado, Luis Amendaño, Daniel Lorences, Alexander Motoche, Danna Zaldumbide, Ozzy Loachamín, Mishel Ramirez, María Mendoza, Eduardo Leines, Elian Gavilanes y Fidel Guacollantes.",
    "Los investigadores del Laboratorio Alan Turing incluyen al Dr. Ángel Valdivieso, Dra. Lorena Barona, Dr. Leonardo Valdivieso, Dr. Patricio Cruz, Dr. Robin Álvarez, y Msc. Pablo Marcillo.",
    "El Laboratorio Alan Turing cuenta con cinco tesistas activos. Los nombres de los tesistas incluyen a Sergio Guamán, Jhonattan Amagua, Christian Nazate, el Ing. Klever Saavedra y Mónica Lucio.",
    "Entre los tesistas activos del laboratorio se encuentran Sergio Guamán, Jhonattan Amagua, Christian Nazate y Mónica Lucio. También colabora el Ing. Klever Saavedra.",
    "Actualmente, hay cinco estudiantes realizando su trabajo de tesis en el laboratorio: Sergio Guamán, Jhonattan Amagua, Christian Nazate, Klever Saavedra y Mónica Lucio.",


    "El enfoque de los proyectos del Laboratorio Alan Turing es el Reconocimiento de Gestos de la Mano utilizando Senales Electromiograficas (EMG) y técnicas avanzadas de Inteligencia Artificial y Machine Learning. Varios proyectos son o fueron dirigidos por el Dr. Marco E. Benalcázar.",
    "La mision del Laboratorio Alan Turing es el Reconocimiento de Gestos de la Mano utilizando Senales Electromiograficas (EMG) y técnicas avanzadas de Inteligencia Artificial y Machine Learning. Varios proyectos son o fueron dirigidos por el Dr. Marco E. Benalcázar.",
    "Normalmente el laboratorio cuenta con equipo de MYO Armbands para realizar los trabajos de Senales Electromiograficas (EMG)",
    "El enfoque principal de los proyectos del Laboratorio Alan Turing es el Reconocimiento de Gestos de la Mano utilizando Senales Electromiográficas (EMG) e Inteligencia Artificial. Los proyectos también abordan el control autonomo de robots UR5.",

    "Un proyecto se dedica al desarrollo de sistemas de reconocimiento de 11 gestos estáticos y dinámicos de la mano, combinando señales EMG con Unidades de Medida Inercial (IMU). Como aplicación, se desarrollaron interfaces Humano-Máquina (para videojuegos) y Humano-Humano (para control de movimiento).",
    "Otro proyecto se enfoca en el desarrollo de modelos generales y específicos por usuario para el reconocimiento de gestos con EMG. El objetivo es crear un protocolo de evaluación riguroso para los modelos de reconocimiento.",
    "Actualmente, el Laboratorio está realizando el proyecto 'Análisis de la influencia de la variación intrapersonal de señales electromiográficas' (PIGR 22 10). Esta investigación busca garantizar que el desempeño de los modelos de reconocimiento de gestos se mantenga constante a lo largo del tiempo, a pesar de las variaciones fisiológicas naturales en las señales EMG.",
    "Los proyectos de clasificación de Señales Electromiográficas (EMG) abordan problemas clave de la literatura, como la cantidad limitada de gestos reconocidos y la sobreestimación de la exactitud de clasificación debido al uso de pocas muestras de testeo. Se propone desarrollar nuevos modelos adaptables.",
    "Los proyectos que se han realizado en el Laboratorio Alan Turing son 'Reconocimiento de Gestos de la Mano Usando Señales Electromiográficas (EMG) e Inteligencia Artificial.', 'Reconocimiento de Gestos de la Mano Usando Señales Electromiográficas e Inteligencia Artificial y su Aplicación para la Implementación de Interfaces Humano-Máquina y Humano-Humano.', 'Clasificación de Señales Electromiográficas del Brazo Humano usando Técnicas de Reconocimiento de Patrones y Machine Learning.', 'Análisis de la influencia de la variación intrapersonal de señales electromiográficas sobre la exac-titud de modelos de reconocimientos de gestos de la mano', 'Interfaz humano humano'",
    "Las tesis de doctorado se centran fuertemente en el Aprendizaje por Refuerzo (RL). Específicamente, se investiga el desarrollo de algoritmos de RL basados en Policy Gradient para el control de prótesis mioeléctricas de mano, con tesis de Mario Moreno y Carlos Ayala.",
    "Algunas tesis de maestria y doctorado realizadas en el Laboratorio Alan Turing son 'Mechanistic interpretability using Sparse Autoencoders in Deep reinforment fully observable and deterministic environments pot Jonathan Zea, MSc.', 'Desarrollo de algoritmos de Aprendizaje por Refuerzo y mejora de las funciones de recompensa en el control de prótesis mioeléctrica de mano por Mario Moreno, MSc.', 'Analysis and Mitigation of Performance Degradation in Machine Learning Models for Hand Gesture Recognition Using sEMG Signals por Gabriel Macías, MSc.', 'Development of Reinforcement Learning algorithms based on Policy Gradient for the operation of myoelectric prostheses por Carlos Ayala, MSc.'",
    "Otros temas de tesis de doctorado incluyen la interpretación mecanicista usando Sparse Autoencoders en entornos de Aprendizaje por Refuerzo (tesis de Jonathan Zea). También se investiga la mitigación del deterioro del desempeño en modelos de reconocimiento de gestos de mano basados en sEMG (tesis de Gabriel Macías).",
    "Los temas principales de las tesis de doctorado son el control de prótesis mioeléctricas mediante Aprendizaje por Refuerzo, y el estudio de la degradación e interpretación de modelos de Machine Learning.",

    # Info integrantes
    "El Dr. Marco E. Benalcázar es el Director del Laboratorio Alan Turing y es catedrático e investigador del Departamento de Informática y Ciencias de la Computación de la EPN. Posee un Doctorado en Ingeniería Electrónica por la Universidad Nacional de Mar del Plata (Argentina).",
    "El Dr. Marco Benalcázar ha dirigido varios proyectos de investigación sobre fundamentos y aplicaciones de inteligencia artificial. Sus áreas clave de estudio son el procesamiento de imágenes médicas y el reconocimiento de gestos de la mano.",

    "La Dra. Lorena Barona obtuvo su título de Ingeniera en Electrónica y Redes de Información en la EPN (2010), su Maestría en Ingeniería de Redes y Servicios Telemáticos por la Universidad Politécnica de Madrid (2013), y su Doctorado en Ingeniería Informática por la Universidad Complutense de Madrid (2017).",
    "Actualmente, la Dra. Lorena Barona es docente en pregrado y maestría en la Facultad de Sistemas de la EPN. Previamente, fue investigadora en la Universidad de Aveiro y en Altice Labs (Portugal), y ha participado en proyectos nacionales y europeos (EU-H2020).",

    "El Dr. Leonardo Valdivieso es docente a tiempo completo en pregrado, maestría y doctorado en la Facultad de Sistemas de la EPN. Posee un Doctorado en Ingeniería Informática por la Universidad Complutense de Madrid y un Máster en Tecnologías de Información por la Hochschule Mannheim (Alemania).",
    "Los campos de investigación actuales del Dr. Leonardo Valdivieso incluyen: Inteligencia Artificial, Sistemas Distribuidos, IoT, Redes de Información y Ciberseguridad. Su experiencia profesional abarca proyectos de investigación nacionales y europeos (EU-H2020).",

    "El Dr. Patricio Cruz es Profesor Titular en el Departamento de Automatización y Control Industrial de la EPN. Recibió su Doctorado en Ingeniería y un Master en Ciencias en Ingeniería Eléctrica de la Universidad de Nuevo México (UNM), en Albuquerque, Estados Unidos.",
    "El Dr. Patricio Cruz se ha especializado en la coordinación de sistemas multi-agentes, el control de vehículos aéreos no tripulados (UAVs) y el desarrollo de plataformas robóticas enfocadas a la rehabilitación de extremidades.",

    "Dr. Robin Alvarez Nació el 2 de abril de 1969 en Cayambe - Ecuador, Ingeniero en Electrónica y Telecomunicaciones, Master en Telecomunicaciones, Doctor en Telecomunicaciones, Postdoctorado (Universidad de California – Berkeley, USA).",

    "El MSc. Pablo Marcillo obtuvo su título de Ingeniero en Sistemas y Ciencias de la Computación y una Maestría en Redes de Comunicación en la Pontificia Universidad Católica del Ecuador (PUCE). Actualmente, está cursando un doctorado en Informática en la Escuela Politécnica Nacional.",

    "El MSc. Jonathan A. Zea es Asistente de Investigación en el Laboratorio Alan Turing en la EPN. Es Ingeniero en Electrónica y Control y tiene una Maestría en Ciencias de la Computación con especialización en Sistemas Inteligentes. Actualmente, sigue un doctorado en Informática centrándose en el tema de alineamiento de Inteligencia Artificial.",
    "El MSc. Jonathan A. Zea cuenta con experiencia como investigador en el Instituto de Ciencia y Tecnología de Gwangju (Corea) y como estudiante de intercambio en la Universidad de Tecnología de Swinburne (Australia).",

    "El MSc. Mario Moreno es Ingeniero en Sistemas Informáticos y Magíster en Computación con mención en Sistemas Inteligentes. Es un miembro activo del Laboratorio de Inteligencia Artificial en el Departamento de Informática y Ciencias de la Computación de la EPN.",

    "El MSc. Carlos Ayala es un asistente de investigación en el Laboratorio Alan Turing y posee una Maestría en Ciencias de la Computación con especialización en Sistemas Inteligentes. Actualmente, cursa un Doctorado en Informática en la EPN.",
    "El MSc. Carlos Ayala centra su investigación doctoral en el desarrollo de algoritmos para la operación de prótesis mioeléctricas. Sus principales áreas de interés son la Ciencia de Datos, Inteligencia Artificial y Machine Learning.",

    "El MSc. Gabriel Macías es un estudiante de doctorado en Ciencias de la Computación con sede en Quito y se graduó como Ingeniero en Sistemas Informáticos y de Computación en la EPN en 2020. Actualmente, desempeña el cargo de Asistente de Investigación de Tiempo Completo en el Departamento de Informática y Ciencias de la Computación.",

    "Pablo Maldonado es estudiante de Ingeniería en Software en la Escuela Politécnica Nacional. Muestra un profundo interés en la Inteligencia Artificial (IA) y el Machine Learning (ML), buscando activamente aplicar estos conocimientos en proyectos del laboratorio.",

    "Luis Amendaño es Tecnólogo Electromecánico y actualmente está en proceso de obtener su título de Ingeniero en Electrónica y Automatización en la Escuela Politécnica Nacional.",
    "La formación de Luis Amendaño combina una sólida base técnica en electromecánica con estudios avanzados en electrónica y automatización, preparándolo para enfrentar desafíos complejos en el campo de la ingeniería.",

    "Daniel Lorences es estudiante de Ingeniería en Ciencias de la Computación en la Escuela Politécnica Nacional. Durante su formación académica, ha adquirido sólidas bases de programación, centrándose principalmente en lenguajes como C# y Python.",

    "Alexander Motoche es estudiante de la EPN, en su quinto semestre. Ha trabajado en el desarrollo de videojuegos interactivos para el IGM y actualmente está involucrado en el desarrollo de una aplicación para el Banco de Alimentos. Le interesa la ciberseguridad y el análisis de datos.",

    "Danna Zaldumbide es estudiante de 6to semestre de Computación, y le atrae la idea de la inteligencia artificial y los proyectos relacionados que puedan contribuir al crecimiento tecnológico y al bienestar de la comunidad.",

    "Ozzy Loachamín es estudiante de Ingeniería en Ciencias de la Computación en la Escuela Politécnica Nacional. Actualmente, forma parte activa del Laboratorio Alan Turing, participando en proyectos de Ciencia de Datos e Inteligencia Artificial que aplica de manera práctica en sus pasantías.",

    "Mishel Ramirez es estudiante de 6to semestre en la carrera de Computación, esta interesada en profundizar los conocimientos inteligencia artificial, su impacto en la innovación tecnológica y sus diversas aplicaciones.",

    "María Mendoza tiene un fuerte interés en el campo de la Inteligencia Artificial y la Ciberseguridad, buscando desarrollarse a un nivel avanzado en ambas. Su dominio del inglés es una habilidad clave que utiliza para acceder a recursos internacionales y aprender sobre las últimas tendencias y herramientas en estas áreas.",

    "Eduardo Leines es estudiante de Ingeniera de Sistemas en Computación de la EPN. Sus enfoques principales son la Inteligencia Artificial y el desarrollo de aplicaciones web (tanto en backend como en frontend). A lo largo de su carrera, ha desarrollado soluciones prácticas que combinan aplicaciones web y programas que funcionan mediante IA.",

    "Elian Gavilanes es pasante es un estudiante de séptimo semestre de Ingeniería en Ciencias de la Computación en la EPN. Su interés principal radica en la Inteligencia Artificial, el aprendizaje automático y la seguridad informática. Su objetivo es aportar activamente en los proyectos de investigación aplicada del laboratorio.",

    "Sergio Guaman es Estudiante de Ciencias de la Computación en la Escuela Politécnica Nacional y forma parte del Laboratorio de Investigación de Inteligencia y Visión Artificial.",

    "Jhonattan Amagua es aspirante a colaborador que actualmente cursa Ingeniería de Software en la EPN. Le apasiona el mundo de la investigación y el campo de la Inteligencia Artificial (IA). Está motivado a aprender más sobre IA y tecnología para poder aportar a estos campos emergentes.",

    "Christian Nazate es estudiante de la carrera de Ingeniería de Software en la Escuela Politécnica Nacional. Ha desarrollado diversas aplicaciones móviles y web con los conocimientos de ingeniería adquiridos a lo largo de sus estudios universitarios. Actualmente realiza pasantías en Billusos como desarrollador web y móvil.",

    "Ing. Klever Saavedra actualmente esta buscando una oportunidad que aproveche su sólida base en mecánica y, al mismo tiempo, le permita aplicar su experiencia en informática y robótica. Le apasiona el campo de la robótica y la Industria 4.0, y está ansioso por cerrar la brecha entre los sistemas mecánicos y la automatización avanzada.",

    "Monica Lucio es una estudiante con interés en la tecnología, el desarrollo de software y la inteligencia artificial. Ha participado en proyectos académicos enfocados en soluciones digitales, seguridad informática y el uso responsable de la tecnología.",

    "Para postularse como pasante en el Laboratorio Alan Turing, el interesado debe tener ganas de trabajar, y enviar su Curriculum Vitae y Academico de la EPN, correo electrónico y número de contacto para iniciar al correo laboratorio.ia@epn.edu.ec.",

    "Para preguntas sobre pasantía o tesis, se debe contactar al MSc. Carlos Ayala. Su correo electrónico es carlos.ayala01@epn.edu.ec.",

    "Para averiguar sobre una pasantia o tesis, el primer paso es enviar un correo electrónico con el asunto claro al correo: laboratorio.ia@epn.edu.ec. Alternativamente, puede comunicarse directamente con el MSc. Carlos Ayala a su correo: carlos.ayala01@epn.edu.ec.",

    # "La ubicacion fisica del personal del laboratorio es: Cubiculo 1 (MSc. Carlos Ayala), Cubiculo 2 (MSc. Gabriel Macias) y Cubiculo 3 (MSc. Jonathan Zea). Las oficinas corresponden a la Dra. Lorena Barona, el Dr. Leonardo Valdivieso y el Dr. Marco Benalcázar.",

    "El Laboratorio Alan Turing tiene publicaciones registradas desde el año 2018 hasta 2024, con un total de 47 publicaciones científicas, la mayoría enfocadas en el Reconocimiento de Gestos de la Mano usando EMG.",
    "Los años con mayor producción científica del Laboratorio Alan Turing fueron 2023 (9 publicaciones) y 2021 (13 publicaciones).",
    "Los temas de publicación más comunes del Laboratorio Alan Turing son: Reconocimiento de Gestos, Machine Learning, Visión Artificial, y Aprendizaje por Refuerzo (Reinforcement Learning).",
    # --- 2024 (Énfasis en Aplicación y Post-procesamiento) ---
    "El Laboratorio Alan Turing publicó dos papers clave en el año 2024. Estos trabajos se enfocan en Machine Learning para el Reconocimiento de Gestos de la Mano (HGR). Los títulos son: 1) 'Hand Gesture Recognition Applied to the Interaction with Video Games' (L. I. Barona López, C. I. León Cifuentes, et al.) y 2) 'CNN-LSTM and post-processing for EMG-based hand gesture recognition' (L. I. Barona López, F. M. Ferri, et al.).",

    # --- 2023 (Énfasis en Feature Selection y Redes Q-Networks) ---
    # "El Laboratorio Alan Turing publico en el año 2023 el paper 'Analysis and Evaluation of Feature Selection and Feature Extraction Methods' de (R. E. Nogales and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2023 el paper 'Development of a Hand Gesture Recognition Model Capable of Online Readjustment Using EMGs and Double Deep-Q Networks' de (D. Díaz, M. E. Benalcázar, L. Barona, and Á. L. Valdivieso).",
    # "El Laboratorio Alan Turing publico en el año 2023 el paper 'Sensor Systems for Gesture Recognition' de (G. Saggio, G. Saggio, and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2023 el paper 'Effects on Hand Gesture Recognition Accuracy after Severe Cranial Trauma' de (J. A. Zea, L. G. M. Santillán, L. I. B. López, Á. L. V. Caraguay, and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2023 el paper 'A comparison of EMG-based hand gesture recognition systems based on supervised and reinforcement learning' de (J. P. Vásconez, L. I. Barona López, Á. L. Valdivieso Caraguay, and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2023 el paper 'Recognition of Hand Gestures Based on EMG Signals with Deep and Double-Deep Q-Networks' de (Á. L. Valdivieso Caraguay, J. P. Vásconez, L. I. Barona López, and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2023 el paper 'Hand Gesture Recognition Using Automatic Feature Extraction and Deep Learning Algorithms with Memory' de (R. E. Nogales and M. E. Benalcázar).",
    "En 2023, el Laboratorio Alan Turing publicó tres papers clave centrados en Aprendizaje por Refuerzo (RL) y redes Q-Networks. Los títulos son: 1) 'Development of a Hand Gesture Recognition Model Capable of Online Readjustment Using EMGs and Double Deep-Q Networks' (D. Díaz, M. E. Benalcázar, et al.), 2) 'A comparison of EMG-based hand gesture recognition systems based on supervised and reinforcement learning' (J. P. Vásconez, L. I. Barona López, et al.) y 3) 'Recognition of Hand Gestures Based on EMG Signals with Deep and Double-Deep Q-Networks' (Á. L. Valdivieso Caraguay, J. P. Vásconez, et al.).",
    "Otros trabajos importantes de 2023 se enfocaron en metodología y sistemas de sensores. Estos incluyen: 1) 'Analysis and Evaluation of Feature Selection and Feature Extraction Methods' (R. E. Nogales and M. E. Benalcázar), 2) 'Hand Gesture Recognition Using Automatic Feature Extraction and Deep Learning Algorithms with Memory' (R. E. Nogales and M. E. Benalcázar) y 3) el estudio 'Sensor Systems for Gesture Recognition' (G. Saggio, G. Saggio, and M. E. Benalcázar).",
    "El total de publicaciones del Laboratorio Alan Turing en 2023 fue de 7. Una investigación destacada fue el paper 'Effects on Hand Gesture Recognition Accuracy after Severe Cranial Trauma' de (J. A. Zea, L. G. M. Santillán, L. I. B. López, et al.), que estudia la precisión del reconocimiento después de un trauma severo.",
    # --- 2022 (Énfasis en Sensores Múltiples) ---
    # "El Laboratorio Alan Turing publico en el año 2022 el paper 'Hand Gesture Recognition Using EMG-IMU Signals and Deep Q-Networks' de (J. P. Vásconez, L. I. Barona López, Á. L. Valdivieso Caraguay, and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2022 el paper 'Analysis of Feature Selection Methods for Hand Gestures Recognition Based on Machine Learning and the Leap Motion Controller' de (R. Nogales and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2022 el paper 'Hand Gesture and Arm Movement Recognition for Multimodal Control of a 3-DOF Helicopter' de (R. Romero et al.).",
    # "El Laboratorio Alan Turing publico en el año 2022 el paper 'An Interface for Audio Control Using Gesture Recognition and IMU Data' de (V. H. Vimos, Á. L. Valdivieso Caraguay, L. I. Barona López, D. Pozo Espín, and M. E. Benalcázar).",
    "En 2022, el Laboratorio Alan Turing publicó trabajos sobre Deep Q-Networks para el reconocimiento de gestos combinando señales EMG e IMU ('Hand Gesture Recognition Using EMG-IMU Signals...'). También publicaron sobre el Control Multimodal de un helicóptero de 3-DOF mediante gestos y movimiento del brazo ('Hand Gesture and Arm Movement Recognition...').",
    "Otras publicaciones clave en 2022 incluyen el Análisis de Métodos de Selección de Características (Feature Selection Methods) para reconocimiento de gestos con Leap Motion Controller. Además, desarrollaron una Interfaz para Control de Audio utilizando reconocimiento de gestos y datos IMU.",

    # --- 2021 (Énfasis en Sistemas Interactivos y Seguridad) ---
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'An interactive system to improve cognitive abilities using electromyographic signals' de (M. E. Benalcázar Palacios et al.).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'A Review of Learning-Based Traffic Accident Prediction Models and Their Opportunities to Improve Information Security' de (P. Marcillo, L. I. Barona López, A. Leonardo, V. Caraguay, and M. Hernández-´ Alvarez).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'Educational Impact on Ecuadorian University Students Due to the COVID-19 Context' de (M. E. Benalcázar, L. Barona, Á. L. Valdivieso, V. H. Vimos, D. Velastegui, and C. J. Santacruz).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'Construction of a Dataset for Static and Dynamic Hand Tracking Using a Non-invasive Environment' de (R. Nogales, M. E. Benalcazar, B. Toalumbo, A. Palate, R. Martinez, and J. Vargas).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'Feature Evaluation of EMG Signals for Hand Gesture Recognition Based on Mutual Information, Fuzzy Entropy and RES Index' de (L. I. Barona-Lopez, A. L. Valdivieso-Caraguay, M. E. Benalcazar, X. Aguas, and J. A. Zea).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'Risk Analysis and Android Application Penetration Testing Based on OWASP 2016' de (T. Borja, M. E. Benalcázar, Á. Leonardo, V. Caraguay, and L. I. Barona López).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'Hand gesture recognition using machine learning and infrared information: a systematic literature review' de (R. E. Nogales and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper '2D Semantic Segmentation of the Prostate Gland in Magnetic Resonance Images using Convolutional Neural Networks' de (S. P. Vacacela and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'A New Methodology for Pattern Recognition Applied to Hand Gestures Recognition Using EMG. Analysis of Intrapersonal and Interpersonal Variability' de (J. A. Ordóñez Flores et al.).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'An Open-Source Data Acquisition and Manual Segmentation System for Hand Gesture Recognition based on EMG' de (J. Zea, M. E. Benalcázar, L. I. Barona López, and Á. L. Valdivieso Caraguay).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'A Brain-Computer Interface for Controlling IoT Devices using EEG Signals' de (K. O. Chicaiza and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'Hand Gesture Recognition and Tracking Control for a Virtual UR5 Robot Manipulator' de (A. Chico et al.).",
    # "El Laboratorio Alan Turing publico en el año 2021 el paper 'A Hand Gesture Recognition System Using EMG and Reinforcement Learning: A Q-Learning Approach' de (J. P. Vásconez, L. I. B. López, Á. L. V. Caraguay, P. J. Cruz, R. Álvarez, and M. E. Benalcázar).",
    "En 2021, el Laboratorio Alan Turing se enfocó en sistemas interactivos y RL. Publicaron sobre el uso de EMG para mejorar habilidades cognitivas y el desarrollo de un sistema de reconocimiento de gestos con Reinforcement Learning (Q-Learning Approach). También crearon un sistema de control y seguimiento para un Robot Manipulador Virtual UR5.",
    "La producción científica de 2021 incluye la construcción de un Dataset para seguimiento de la mano (static and dynamic hand tracking), la evaluación de características de señales EMG (Mutual Information, Fuzzy Entropy), y una revisión sistemática sobre reconocimiento de gestos usando información infrarroja. Adicionalmente, realizaron un análisis de riesgo y Penetration Testing para aplicaciones Android (OWASP 2016).",
    "Otras áreas cubiertas en 2021 fueron el impacto educativo en estudiantes por el contexto COVID-19, el uso de Redes Neuronales para la Segmentación 2D Semántica de la Próstata en resonancias magnéticas, y la creación de una Interfaz Cerebro-Computadora (BCI) para el control de dispositivos IoT usando señales EEG.",

    # --- 2020 (Énfasis en Robustez y Librerías) ---
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'A Novel Technique for Improving the Robustness to Sensor Rotation in Hand Gesture Recognition Using sEMG' de (V. H. Vimos, M. Benalcázar, A. F. Oña, and P. J. Cruz).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'A Survey on Hand Gesture Recognition Using Machine Learning and Infrared Information' de (R. Nogales and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'Sign Language Recognition Using Wearable Electronics: Implementing k-Nearest Neighbors with Dynamic Time Warping and Convolutional Neural Network Algorithms' de (G. Saggio, P. Cavallo, M. Ricci, V. Errico, J. Zea, and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'Real-Time Hand Gesture Recognition Using Surface Electromyography and Machine Learning: A Systematic Literature Review' de (A. Jaramillo-Yánez, M. E. Benalcázar, and E. Mena-Maldonado).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'A User-Specific Hand Gesture Recognition Model Based on Feed-Forward Neural Networks, EMGs, and Correction of Sensor Orientation' de (M. E. Benalcázar, Á. L. V. Caraguay, and L. I. B. López).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'An Energy-Based Method for Orientation Correction of EMG Bracelet Sensors in Hand Gesture Recognition Systems' de (L. I. B. López et al.).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'Real-Time Hand Gesture Recognition Using KNN-DTW and Leap Motion Controller' de (R. Nogales and M. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'Adaptive Non-linear Control for a Virtual 3D Manipulator' de (A. Ona, V. Vimos, M. Benalcazar, and P. J. Cruz).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'A Model for Real-Time Hand Gesture Recognition Using Electromyography (EMG), Covariances and Feed-Forward Artificial Neural Networks' de (M. E. Benalcazar et al.).",
    # "El Laboratorio Alan Turing publico en el año 2020 el paper 'Real-Time Hand Gesture Recognition: A Long Short-Term Memory Approach with Electromyography' de (J. A. Zea and M. E. Benalcázar).",
    "En 2020, el Laboratorio Alan Turing publicó trabajos importantes sobre la Robustez de Sensores y la Corrección de Orientación en el reconocimiento de gestos. Estos incluyen papers sobre 'A Novel Technique for Improving the Robustness to Sensor Rotation in Hand Gesture Recognition Using sEMG' y 'An Energy-Based Method for Orientation Correction of EMG Bracelet Sensors in Hand Gesture Recognition Systems'.",
    "Publicaciones clave de 2020 se centraron en la Revisión Sistemática de Literatura sobre reconocimiento de gestos (usando EMG e información infrarroja) y la creación de nuevos modelos de Deep Learning. Específicamente, papers sobre la aplicación de Long Short-Term Memory (LSTM) y modelos de Redes Neuronales Feed-Forward para reconocimiento de gestos en tiempo real.",
    "El laboratorio también se enfocó en el Control Robótico y la aplicación de interfaces. Publicaron sobre 'Adaptive Non-linear Control for a Virtual 3D Manipulator', el reconocimiento de gestos en lenguaje de señas con Wearable Electronics, y el uso de KNN-DTW y Leap Motion Controller para reconocimiento de gestos en tiempo real.",

    # --- 2019 Y 2018 (Énfasis en Fundamentos) ---
    # "El Laboratorio Alan Turing publico en el año 2019 el paper 'Technical mapping of the grooming anatomy using machine learning paradigms: An information security approach' de (P. Zambrano et al.).",
    # "El Laboratorio Alan Turing publico en el año 2019 el paper 'Real-Time Hand Gesture Recognition Using the Leap Motion Controller and Machine Learning' de (R. Nogales and M. Benalcazar).",
    # "El Laboratorio Alan Turing publico en el año 2019 el paper 'Short-Term Hand Gesture Recognition using Electromyography in the Transient State, Support Vector Machines, and Discrete Wavelet Transform' de (A. Jaramillo-Yanez, L. Unapanta, and M. E. Benalcazar).",
    # "El Laboratorio Alan Turing publico en el año 2019 el paper 'Real-time hand gesture recognition model using deep learning techniques and EMG signals' de (E. A. Chung and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2019 el paper 'Reconocimiento de Gestos de la Mano en Tiempo Real Usando Leap Motion Controller y Machine Learning' de (N. R. Jhon et al.).",
    # "El Laboratorio Alan Turing publico en el año 2019 el paper 'Machine learning for computer vision: A review of theory and algorithms' de (M. E. Benalcázar).",
    "En 2019, el Laboratorio Alan Turing sentó las bases en Reconocimiento de Gestos de la Mano en tiempo real. Se publicaron trabajos utilizando el Leap Motion Controller y señales EMG, incluyendo el uso de Deep Learning y técnicas de Transformada Wavelet Discreta para mejorar la precisión.",
    "Otras publicaciones de 2019 incluyen una revisión exhaustiva sobre la teoría y algoritmos de Machine Learning para Visión por Computadora (M. E. Benalcázar), y una investigación sobre seguridad de la información bajo el título 'Technical mapping of the grooming anatomy using machine learning paradigms'.",

    # "El Laboratorio Alan Turing publico en el año 2018 el paper 'Real-time hand gesture recognition based on artificial feed-forward neural networks and EMG' de (M. E. Benalcázar, C. E. Anchundia, J. A. Zea, P. Zambrano, A. G. Jaramillo, and M. Segura).",
    # "El Laboratorio Alan Turing publico en el año 2018 el paper 'Real-Time Hand Gesture Recognition Based on Electromyographic Signals and Artificial Neural Networks' de (C. Motoche and M. E. Benalcázar).",
    # "El Laboratorio Alan Turing publico en el año 2018 el paper 'Virtual Rehabilitation System for Fine Motor Skills Using a Functional Hand Orthosis' de (M. A. León et al.).",
    # "El Laboratorio Alan Turing publico en el año 2018 el paper 'Optimal clustering under uncertainty' de (L. A. Dalton, M. E. Benalcazar, and E. R. Dougherty).",
    # "El Laboratorio Alan Turing publico en el año 2018 el paper 'Design of a Software Architecture and Practical Applications to Exploit the Capabilities of a Human Arm Gesture Recognition System' de (F. E. Ramirez, M. Segura-Morales, and M. Benalcazar).",
    "En 2018, el Laboratorio Alan Turing sentó las bases publicando sobre el reconocimiento de gestos de la mano en tiempo real utilizando Señales Electromiográficas (EMG) y Redes Neuronales Artificiales (Artificial Feed-forward Neural Networks), siendo un punto de partida fundamental para su área de estudio.",
    "Otras publicaciones clave en 2018 exploraron el Diseño de Arquitecturas de Software para explotar las capacidades del reconocimiento de gestos del brazo humano. También se enfocaron en el desarrollo de un Sistema de Rehabilitación Virtual para habilidades motoras finas usando una ortesis funcional.",

    "El Laboratorio Alan Turing ofrece varios datasets de EMG: EMG-EPN-612 (5 gestos, 612 usuarios), EMG-EPN-120 (5 gestos, 120 usuarios) en formato Matlab, y EMG-IMU-EPN-100+ (12 categorías, incluyendo gestos estáticos y dinámicos, con sensores EMG e IMU).",
    "El dataset EMG-EPN-612 incluye señales EMG de 5 gestos de la mano, con datos de entrenamiento y validación de 612 usuarios. Se puede descargar en formato JSON.",
    "El dataset EMG-IMU-EPN-100+ es un conjunto de datos EMG-IMU para 12 categorías diferentes de gestos con las manos, utilizando los brazaletes Myo y G-force.",

    "El laboratorio desarrolló la aplicación 2021_CTR_UR5_EPN para 'HGR and Tracking Control for a Virtual UR5 Robot', un HMI que usa gestos EMG e IMU para controlar un manipulador virtual UR5 de seis grados de libertad (DOF).",
    "Existe la aplicación 2021_3DOF_HGR_EPN para 'Hand Gesture and Arm Movement Recognition for Multimodal Control of a 3-DOF Helicopter'. Este sistema usa señales EMG e IMU para controlar el movimiento de un helicóptero de rotor dual.",
    "El modelo 2020_DTW_ANN_EPN realiza reconocimiento de gestos de la mano basado en redes neuronales *feed-forward* y está disponible en versiones Python y Matlab. La aplicación 2020_ROT_SVM_EPN se enfoca en la corrección de orientación del sensor EMG con SVM.",

    "El laboratorio proporciona anexos de papers, como el 'SLR Prostate Gland Segmentation', que trata sobre segmentación semántica 2D de la próstata en imágenes de resonancia magnética usando Redes Neuronales.",
    "Un enlace de interés es el del proyecto '2020_SARS_COV_2', que contiene reportes sobre la evolución del número diario de defunciones por cada 100 mil habitantes en 2020.",

    "El Laboratorio Alan Turing fue reconocido en el evento SinergIA Reconocimientos del MINTEL en la categoría 'Salud y Bienestar'. Su proyecto fue el mejor evaluado entre todos los postulantes de todas las categorías.",
    "El Laboratorio Alan Turing consiguio en el evento SinergIA Reconocimientos del MINTEL en la categoría 'Salud y Bienestar'. Su proyecto fue el mejor evaluado entre todos los postulantes de todas las categorías.",
    "El Laboratorio Alan Turing logro en el evento SinergIA Reconocimientos del MINTEL en la categoría 'Salud y Bienestar'. Su proyecto fue el mejor evaluado entre todos los postulantes de todas las categorías.",

    "El Laboratorio Alan Turing presentó trabajos en el 30th International Conference on Artificial Neural Networks (ICANN 2021), incluyendo 'A Hand Gesture Recognition System Using EMG and Reinforcement Learning: A Q-Learning Approach'.",
    "El Laboratorio Alan Turing también presentó en el 2nd International Conference on Education and Artificial Intelligence Technologies (EAIT 2021) un trabajo titulado 'An Interactive System to improve Cognitive Abilities using Electromyography Signals'.",

    "El Laboratorio Alan Turing ofrece el curso 'Percepción Computacional' como parte de la Maestría en Computación de la Escuela Politécnica Nacional, impartido por el Dr. Marco E. Benalcázar.",

    "El Dr. Marco Benalcázar es columnista regular del Diario El Comercio con artículos en 2024 sobre Worldcoin, Chat GPT, acceso a la universidad y regulación de la Inteligencia Artificial ecuatoriana.",
    "El Laboratorio tuvo cobertura en prensa y medios entre 2018 y 2024, incluyendo Diario El Comercio, Diario Expreso y Diario El Universo, discutiendo sobre robótica, sistemas de gestos y aplicaciones de IA.",

    "El Laboratorio Alan Turing busca tecnicos de investigacion con los requisitos mínimos para el puesto de técnico de investigación que son: Título de tercer nivel en computación/sistemas, suficiencia B1/superior en inglés, acreditar 20h en cursos de capacitación sobre IA y análisis de datos, y no estar cursando un doctorado en la EPN.",
    "La postulación al técnico de investigación requiere la entrega física de carpetas del 25 al 29 de septiembre de 2025 en la Secretaría del DICC de la Facultad de Ingeniería de Sistemas.",
    "El Laboratorio de Investigación en Inteligencia y Visión Artificial 'Alan Turing' está ubicado en Quito, Ecuador, en Ladrón de Guevara E11-253, Campus Politécnico 'José Rubén Orellana', Facultad de Ingeniería de Sistemas, Cuarto Piso. El correo es laboratorio.ia@epn.edu.ec.",
    "El Laboratorio Alan Turing siempre está abierto a recibir estudiantes para realizar pasantías o trabajos de tesis en áreas de inteligencia y visión artificial. Los interesados deben enviar un correo electronico ",
    "Actualmente, la única vacante de trabajo formal es la convocatoria para el técnico de investigación (Proceso de Selección 2025). Para otras formas de colaboración, se debe consultar directamente con el Laboratorio.",
    "El Laboratorio Alan Turing está activamente abierto a recibir estudiantes para realizar pasantías de investigación o desarrollar sus trabajos de titulación (tesis) en áreas de Inteligencia y Visión Artificial. Los interesados deben contactar directamente a los investigadores principales para consultar sobre cupos disponibles.",
    "Para iniciar el proceso de pasantias, tesis o vinculacion, la persona interesada debe enviar un correo electrónico a laboratorio.ia@epn.edu.ec con el asunto que desee.",

    "El Laboratorio Alan Turing ahora mismo cuenta con 26 integrantes activos, entre ellos investigadores, colaboradores, pasantes y tesistas, puedes preguntar sobre ellos con mas detalles si deseas",
    "Para iniciar el proceso de pasantías o tesis, debes enviar un correo electrónico con tu solicitud al correo oficial: laboratorio.ia@epn.edu.ec. También puedes contactar al MSc. Carlos Ayala (carlos.ayala01@epn.edu.ec) para dudas sobre el proceso.",

    "Para ingresar como pasante, se requiere tener ganas de trabajar, y enviar tu Curriculum Vitae y número de contacto al correo electronico del laboratorio: laboratorio.ia@epn.edu.ec.",
    "Para realizar tu tesis o pasantía en el Laboratorio Alan Turing, el procedimiento es: enviar un correo electrónico con tu solicitud y CV al email oficial.",
    "El logro más importante del Laboratorio Alan Turing fue ser MEJOR EVALUADO y reconocido con el premio SinergIA Reconocimientos de MINTEL en la categoría 'Salud y Bienestar', destacándose entre todos los proyectos.",
    "El logro más importante del Laboratorio Alan Turing fue obtener el reconocimiento SinergIA del Ministerio de Telecomunicaciones (MINTEL). Fue el MEJOR PROYECTO EVALUADO en la categoría 'Salud y Bienestar', destacando el éxito del laboratorio en proyectos de impacto social.",
    "El Director del Laboratorio Alan Turing es el Dr. Marco E. Benalcázar.",
    "La dirección web principal para visitar o navegar el Laboratorio es: https://laboratorio-ia.epn.edu.ec.",
    "El Laboratorio Alan Turing cuenta con 26 integrantes activos en total, incluyendo al Director, investigadores principales, colaboradores de Doctorado, tesistas y pasantes.",
    "El email del laboratorio es: laboratorio.ia@epn.edu.ec para cualquier cuestion",
    "Pueden contactarse con el email laboratorio.ia@epn.edu.ec. o tambien puede llamar al numero del laboratorio Alan Turing"
]

### Cargar el Modelo de Lenguaje Local

Esta celda cargará lo que es el LLM GPT2 desde la carpeta local, ya que por la restriccion de puertos de la EPN genero problemas el solo poner la direccion URL. Por ello, asegurarse que el path en este caso para jetson nano para el docker creado el path normalmente es '/nvdli-nano/LLMs/...', donde se encuentran todos los modelos tanto de embedding como el LLM. El embedding mapea oraciones y párrafos a un espacio vectorial de una densidad de 384 dimensiones y se puede utilizar para tareas como agrupamiento o búsqueda semántica. Cuando el embedding se lo combina con un modelo de inferencia o instructivo, este puede generar respuestas en base al contexto dado por por el embedding, en este caso el Flan-T5 (large para computadoras con hardware medianamente bueno y small o base para el jetson nano dependiendo que tan bien se ejecute todo). Si se quiere ejecutar en un jetson nano, este debe correr los modelos mas pequeños como los son el small o base, a continuacion lo diferentes parametros de cada modelo:

- Flan-T5 Small: 77 millones de parametros
- Flan-T5 Base: 248 millones de parametros
- Flan-T5 Large: 783 millones de parametros
- Flan-T5 XL: 2.85 billones de parametros
- Flan-T5 XXL: 11.3 billones de parametros

In [2]:
# --- 2. CARGA DE MODELOS (SETUP) ---
try:
    # Selecciona la GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Dispositivo de inferencia seleccionado: {device}")

    print(f"Cargando Modelo LLM ({LLM_MODEL_NAME})...")
    llm_tokenizer = T5Tokenizer.from_pretrained(LLM_MODEL_NAME)
    llm_model = T5ForConditionalGeneration.from_pretrained(LLM_MODEL_NAME).to(device)

    print(f"Cargando Modelo de Embedding ({EMBEDDING_MODEL_NAME})...")
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME).to(device)

except Exception as e:
    print(f"\n[ERROR CRÍTICO DE CARGA] Falló al cargar modelos. Error: {e}")
    llm_model, llm_tokenizer, embedding_model = None, None, None

Dispositivo de inferencia seleccionado: cuda
Cargando Modelo LLM (google/flan-t5-small)...


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Cargando Modelo de Embedding (intfloat/multilingual-e5-small)...


In [20]:
# --- OBTENER EMBEDDING ---
def get_embedding(text):
    # Genera el embedding usando el modelo local.
    if embedding_model is None: raise RuntimeError("Modelo de embedding no cargado.")

    embeddings = embedding_model.encode([text], convert_to_tensor=True)
    return embeddings[0].cpu().numpy().astype('float32')

In [ ]:
# --- OBTENER RESPUESTA POST-RAG---
def generate_llm_response(prompt):
    # Genera la respuesta usando el LLM Flan-T5 con la sintaxis Encoder-Decoder.
    if llm_model is None or llm_tokenizer is None: raise RuntimeError("Modelo LLM no cargado.")

    print(f"[GENERACIÓN] Ejecutando {LLM_MODEL_NAME}...")

    try:
        # Generación
        input_data = llm_tokenizer(prompt, return_tensors="pt", max_length=1024).input_ids.to(device)
        output = llm_model.generate(
            input_data,
            max_new_tokens=120,            # Aumentado para respuestas más completas
            min_length=40,                 # Asegura respuestas con contenido
            do_sample=True,
            early_stopping=True
        )
        generated_text = llm_tokenizer.decode(output[0], skip_special_tokens=True)

        # POST-PROCESAMIENTO
        final_text = generated_text

        # Intentar cortar después del marcador 'Respuesta:'
        #response_marker = "Respuesta:"
        #if response_marker in final_text:
             # Si lo encontramos, cortamos el texto de instrucción ANTES.
             #final_text = final_text.rsplit(response_marker, 1)[-1].strip()
        #else:
             # Si no lo encontramos, asumimos que TODO el texto generado es la respuesta.
             #pass

        final_text = final_text.replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace("?",'').strip()
        # Limpieza de palabras residuales
        cleaned_output = re.sub(r'[^A-Za-z0-9\s\.\,\?\!\:\;@\-\áéíóúÁÉÍÓÚñÑ]+', '',  final_text, flags=re.IGNORECASE)

        # Manejo de Fallback (si sale vacio)
        if not cleaned_output or len(cleaned_output.strip()) < 2 in cleaned_output.lower():
             return "Dato no disponible." # Forzamos el mensaje de fallback en español

        return final_text

    except Exception as e:
        return f"[ERROR DE INFERENCIA EN EL LLM]: {e}"

In [66]:
# --- CREAR FAISS CON EL KNOWLEDGE---
def load_or_create_knowledge_base():
    #Carga los archivos FAISS y Chunks, o los crea desde la lista manual, guardándolos en el directorio actual.

    # CARGAR ARCHIVOS GUARDADOS
    # Se usan solo los nombres de archivo, asumiendo que están en el directorio actual (os.getcwd()).
    if os.path.exists(INDEX_NAME) and os.path.exists(CHUNKS_NAME):
        try:
            print(f"\n[INFO] Archivos de conocimiento encontrados. Cargando {INDEX_NAME}...")
            faiss_index = faiss.read_index(INDEX_NAME)
            all_chunks = np.load(CHUNKS_NAME, allow_pickle=True).tolist()
            return faiss_index, all_chunks
        except Exception:
            pass

    # CREACIÓN CON DATOS MANUALES
    print("\n[INFO] Iniciando CREACIÓN con datos manuales.")

    if not knowledge_chunks_manual or llm_model is None:
        print("[ERROR] Modelos o datos manuales vacíos. Imposible crear base.")
        return None, None

    print(f"-> Generando embeddings para {len(knowledge_chunks_manual)} fragmentos.")

    all_chunks = knowledge_chunks_manual
    embeddings = [get_embedding(chunk) for chunk in all_chunks]
    embeddings_np = np.array(embeddings, dtype='float32')

    dimension = embeddings_np.shape[1]
    faiss_index = faiss.IndexFlatL2(dimension)
    faiss_index.add(embeddings_np)

    # Guardado local
    try:
        faiss.write_index(faiss_index, INDEX_NAME)
        np.save(CHUNKS_NAME, np.array(all_chunks, dtype=object))
        print("[ÉXITO] Índice FAISS creado y guardado.")
    except Exception as e:
        print(f"[ERROR DE GUARDADO] Falló: {e}")

    return faiss_index, all_chunks

faiss_index, all_chunks = load_or_create_knowledge_base()


[INFO] Iniciando CREACIÓN con datos manuales.
-> Generando embeddings para 125 fragmentos.
[ÉXITO] Índice FAISS creado y guardado.


#### Funcion generar respuesta junto el embedding

En esta parte se realiza lo que seria generacion de respuesta dado un mensaje del usuario y el contexto

In [92]:
SIMILARITY_THRESHOLD = 0.25

def flan_predict(query: str):
    """Flujo RAG completo: Búsqueda, prompt y generación."""
    if llm_model is None or faiss_index is None:
        return "El LLM o índice RAG no están cargados."

    # Búsqueda de fragmentos (Retrieval)
    query_vector = get_embedding(query)
    distances, indices = faiss_index.search(query_vector.reshape(1, -1), k=3)
    relevant_chunks = [all_chunks[i] for i in indices[0].tolist()]
    context = "\n".join(relevant_chunks)

    if distances[0][0] > SIMILARITY_THRESHOLD:
        return "No tengo información relevante para responder a esa pregunta. Por favor, reformule su consulta."
    
    # Construcción del Prompt
    prompt = f"""You are a concise assistant for the Alan Turing Laboratory.

    Context: {context}

    Question: {query}

    Answer in Spanish, with large and detailed information and do not response with the question again:"""

    # 3. Llamada al LLM
    generated_text = generate_llm_response(prompt)

    # Convertir la pregunta original a una versión limpia y sin espacios para un corte robusto
    clean_query_start = re.sub(r'[^A-Za-z0-9\s]+', '', query.lower()) 

    final_response = generated_text

    # 1. Cortar el residuo largo de instrucción
    if "Respuesta:" in final_response:
        final_response = final_response.rsplit("Respuesta:", 1)[-1].strip()
        
    # 2. Limpieza final de la pregunta que el LLM repitió al inicio:
    if final_response.lower().startswith(clean_query_start[:10]):
        # Si el inicio de la respuesta coincide con el inicio de la pregunta, es un eco.
        # Quitamos la pregunta original (ya limpia) del inicio de la respuesta.
        final_response = final_response.replace(query, '', 1).strip()

    # 3. Limpieza de caracteres no deseados
    final_response = re.sub(r'[^A-Za-z0-9\s\.\,\?\!\:\;@\-\(\)áéíóúÁÉÍÓÚñÑ]+', '', final_response, flags=re.IGNORECASE)
    return final_response.strip()

## Chatbot

El chatbot se contruyo a base de la libreria ipywidget, ya que gradio genero problemas en cuanto a compatibilidad con la version 3.6 de python que jetson nano probe en la imagen jetpack 4.6, esto mismo es para versiones posteriores, por ello se opto de realizar el chatbot con esta libreria que resulto casi similar a gradio a excepcion de la ventaja de gradio de abrir un pequeño entorno web y interfaces mas estilizadas que nos puede proporcionar este ultimo

#### Estilo
Aqui se creo toda la parte de estilos para los diferentes widgets que se crearan en el chatbot

In [24]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Clases que usaremos en ipywidgets para darle un estilo
custom_css = """
/* === ESTRUCTURA Y LAYOUT === */
.main_container_ipy {
    max-width: 1200px; margin: auto; padding: 20px; border-radius: 15px;
    background: linear-gradient(135deg, #3a7bd5, #112240);
}
#header_container_ipy {
    display: flex;
    flex-direction: column; /* Apila elementos verticalmente */
    align-items: center;
    margin-bottom: 20px;
    padding: 20px 0; /* Espacio interno para el header */
}
/* Estilo específico para la imagen de cabecera que combina los logos */
#header_container_ipy .combined-banner {
    width: 400px; /* Tamaño que mantiene la calidad y se centra */
    max-width: 90%;
    height: auto;
    margin-bottom: 20px;
}
/* === TEXTOS Y TÍTULOS === */
#chatbot_title_ipy h1 {
    text-align: center; color: #ffffff; font-size: 2.5em; font-weight: bold; margin-bottom: 5px;
}
#chatbot_description_ipy p {
    text-align: center; color: #ced4da; margin-bottom: 0; font-size: 1.1em;
}

/* === CONTENEDOR DEL CHAT === */
#chat_container_ipy {
    border: 1px solid #FFC72C; border-radius: 10px; padding: 15px;
    background-color: rgba(29, 53, 87, 0.5);
    height: 450px; overflow-y: scroll; font-family: sans-serif;
}
.message-row { display: flex; align-items: flex-start; margin-bottom: 10px; }
.avatar-container { width: 40px; height: 40px; border-radius: 50%; overflow: hidden; margin-right: 10px; flex-shrink: 0; margin-top: 8px;}
.avatar-container img { width: 100%; height: 100%; object-fit: contain; }
.message-bubble {
    padding: 10px 15px; border-radius: 18px; max-width: 85%;
    line-height: 1.4; white-space: pre-wrap;
    animation: bubble-fade-in 0.4s ease-out;
}
@keyframes bubble-fade-in {
    from {
        opacity: 0;
        transform: translateY(10px);
    }
    to {
        opacity: 1;
        transform: translateY(0); /* Termina en su posición normal */
    }
}
.user-bubble { background-color: #007bff; color: white; border-bottom-left-radius: 0; margin-left: auto; }
.bot-bubble { background-color: #FFC72C; color: #000000; border-bottom-right-radius: 0; }

/* Animacion de los 3 puntos mientras genera respuesta */
.bot-bubble.thinking-message {
    width: 75px;
    height: 40px;
    padding: 0 !important;
    display: flex;
    align-items: center;
    justify-content: center;
}

.typing-indicator { display: flex; align-items: center; justify-content: center; }
.typing-indicator span {
    height: 9px; width: 9px; background-color: #000000;
    opacity: 0.9; border-radius: 50%; margin: 0 3px;
    animation: bounce 1.2s infinite ease-in-out;
}
.typing-indicator span:nth-of-type(1) { animation-delay: -0.4s; }
.typing-indicator span:nth-of-type(2) { animation-delay: -0.2s; }
.typing-indicator span:nth-of-type(3) { animation-delay: 0s; }
@keyframes bounce {
  0%, 80%, 100% { transform: scale(0); }
  40% { transform: scale(1.0); }
}

/* === BOTONES Y ENTRADAS === */
.prompt-buttons button {
    background: rgba(255, 255, 255, 0.05) !important; border: 1px solid #FFC72C !important;
    color: #ffffff !important;; font-size: 0.9em !important; border-radius: 10px !important;

    white-space: normal !important;
    height: auto !important;
    min-height: 40px;               /* Asegura una altura mínima consistente */
    padding-top: 8px;               /* Añade un poco de espacio vertical */
    padding-bottom: 8px;
    line-height: 1.2;
    transition: all 0.2s ease-in-out;
}
.prompt-buttons button:hover { background: #FFC72C !important; color: #0a192f !important; }

/* Estilo para el cuadro de texto donde el usuario escribe */
.widget-text input[type="text"] {
    border: 1px solid #4a5568;         /* Borde gris azulado */
    background-color: #2c3e50;       /* Fondo azul oscuro más claro */
    color: white !important;
    border-radius: 8px;                /* Esquinas redondeadas */
    padding: 0px 10px;                 /* Espacio interno para que el texto no pegue al borde */
}
/* Efecto de resplandor amarillo en cuadros de texto */
.widget-text input[type="text"]:focus {
    border-color: #FFC72C !important;
    box-shadow: 0 0 5px 1px #ffc107 !important;
}

.widget-text input[type="text"]::placeholder {
    color: #ced4da !important;
    opacity: 0.8;              /* no desvanecido */
}

.send-button, .clear-button {
    border-radius: 8px !important;
    transition: filter 0.2s ease-in-out; /* Añade la animación */
}

.send-button {
    background-color: #007bff !important; color: white !important;
    border-radius: 8px !important;
}
.clear-button {
    background-color: #6c757d !important; color: white !important;
    border-radius: 8px !important;
}

/* Efecto hover para los botones */
.send-button:hover, .clear-button:hover {
    filter: brightness(0.9); /* Oscurece al pasar el mouse */
}
/* --- Estilo para el ícono de ayuda que muestra informacion (?) --- */
.tooltip-trigger {
    display: inline-block;
    width: 22px;
    height: 22px;
    line-height: 22px;          /* Centra verticalmente el signo de interrogación */
    text-align: center;
    border: 1.5px solid #ced4da;  /* Borde para formar el círculo */
    border-radius: 50%;         /* Redondea el borde para hacerlo un círculo perfecto */
    font-weight: bold;
    color: #ced4da;
    cursor: help;               /* Cambia el cursor del mouse a uno de 'ayuda' */
    transition: all 0.2s ease-in-out; /* Añade una transición suave para el efecto hover */
    margin-right: 8px;          /* Espacio entre el ícono y el texto */
}

/* --- Estilos para el panel de información desplegable --- */
.info-panel {
    background-color: rgba(17, 34, 64, 0.9);
    border: 1px solid #FFC72C;
    border-radius: 10px;
    padding: 20px;
    margin: 10px 0;
    overflow: hidden; /* Oculta el contenido durante la animación */
    transition: all 0.4s ease-in-out; /* animacion*/
}

/* Estilos para el contenido de texto dentro del panel */
.info-panel-text h3 {
    margin-top: 0;
    color: #FFC72C !important;
}
.info-panel-text h4 {
    color: #ffffff !important;
}
.info-panel-text p, .info-panel-text li {
    line-height: 1.6;
    color: #ced4da !important;
}
.info-panel-text ul {
    padding-left: 25px;
    margin-bottom: 0;
}

/* Efecto al pasar el mouse sobre el ícono */
.tooltip-trigger {
    display: inline-block;
    width: 24px; height: 24px; line-height: 24px;
    text-align: center;
    border-radius: 50%;
    font-weight: bold;
    background-color: #FFC72C;
    color: #112240;
    cursor: pointer;
    transition: filter 0.2s ease-in-out;
    margin-left: 8px;
    border: none;
}
.tooltip-trigger:hover {
    filter: brightness(0.9);
}

/* --- Estilos para el pie del Footer --- */
#footer_ipy {
    text-align: center;
    margin-top: 25px;
    border-top: 1px solid #4a5568;
    padding-top: 15px;
}
#footer_ipy .lab-link {
    color: #ced4da;
    text-decoration: none;
    font-size: 1em;
    display: block;
    margin-bottom: 10px;
    transition: color 0.2s ease-in-out;
}
#footer_ipy .lab-link:hover {
    color: #FFC72C;
    text-decoration: underline;
}
#footer_ipy .social-links a {
    color: #ced4da;
    text-decoration: none;
    margin: 0 10px;
    transition: color 0.2s ease-in-out;
}
#footer_ipy .social-links a:hover {
    color: #FFC72C;
}
"""

# aplicar CSS
display(HTML(f"<style>{custom_css}</style>"))


#### Desplegar la interfaz

En esta parte desplegamos el chatbot

In [ ]:
# Se importa la librería 'base64' para poder convertir los archivos de imagen a texto.
# Esto nos permite poner las imágenes directamente en el código HTML
import base64

# --- VARIABLES DE HISTORIAL ---
# Guardamos cada mensaje (tanto del usuario como del bot) aquí en formato HTML.
chat_history_html = []


# --- WIDGETS DE LA INTERFAZ ---
# En esta sección, creamos todas las partes visuales

# Cabecera con logos y títulos.
# Usamos un bloque 'try..except' para manejar el error en caso de que las imagenes usadas no se encuentren
# en la misma carpeta que el notebook.
try:
    # Se abre el archivo de la imagen en modo de lectura binaria ('rb').
    with open("LOGO_Facultades-es.png", "rb") as f:
        # Se lee el archivo y se codifica en base64, para luego convertirlo en un string.
        banner_b64 = base64.b64encode(f.read()).decode("utf-8")
    with open("Logo IA LAB.png", "rb") as f:
        avatar_b64 = base64.b64encode(f.read()).decode("utf-8")

    # Se crea un widget HTML. Usamos f-strings (f"...") para poder insertar nuestras variables
    # (como el texto base64 de la imagen) directamente dentro del código HTML.
    header_html = widgets.HTML(f"""
        <div id="header_container_ipy">
            <img src="data:image/png;base64,{banner_b64}" class="combined-banner" alt="Logo de la EPN y Laboratorio Alan Turing">
            <div id="chatbot_title_ipy"><h1>Asistente del Laboratorio Alan Turing</h1></div>
            <div id="chatbot_description_ipy"><p>Impulsado por RAG y modeloLLM.</p></div>
        </div>
    """)
except FileNotFoundError:
    # Si las imágenes no se encuentran, mostramos un error claro.
    header_html = widgets.HTML("<h1>Error: Asegúrate de que los archivos 'LOGO_Facultades-es.png' y 'Logo IA LAB.png' estén en la misma carpeta.</h1>")

# --- Panel de información (inicialmente oculto) ---
info_panel_content = widgets.HTML("""
    <h3>Acerca del Chatbot</h3>
    <h4>Arquitectura RAG (Recuperación Aumentada)</h4>
    <p>Este asistente utiliza la arquitectura <strong>RAG</strong> para responder preguntas basándose exclusivamente en la base de conocimiento estructurada del Laboratorio Alan Turing.</p>
    <ul>
        <li><strong>LLM (Modelo de Inferencia):</strong> Flan-T5 (large para hardware mediano o small para Jetson Nano).</li>
        <li><strong>Modelo de Embedding:</strong> Se utiliza <strong>paraphrase-multilingual-MiniLM-L12-v2</strong>, un modelo altamente eficiente para la búsqueda semántica en una base de datos (FAISS).</li>
        <li><strong>Base de Conocimiento:</strong> Los datos provistos por la pagina del Laboratorio Alan Turing e indexados en <strong>FAISS</strong>.</li>
        <li><strong>Misión:</strong> Proporcionar respuestas lo mas precisas y libres segun la informacion adquirida.</li>
    </ul>
    <h4>Nota de Rendimiento</h4>
    <p>El modelo puede ser lento si no se usa una GPU dedicada. En entornos de CPU o la Jetson Nano, la velocidad de generación puede estar limitada incluso con las versionas mas ligeras.</p>
""")

info_panel_content.add_class("info-panel-text")

close_button = widgets.Button(description="Cerrar")
close_button.add_class("clear-button")

# El panel de informacion de GPT-2 ahora se oculta cambiando la opacidad, altura y visibilidad.
info_panel = widgets.VBox(
    [info_panel_content, close_button],
    layout={'visibility': 'hidden', 'opacity': '0', 'max_height': '0px', 'padding': '0'}
)
info_panel.add_class("info-panel")


# Boton de ayuda con signo de interrogacion
help_button = widgets.Button(description="?", layout={'width': '40px', 'height': '40px'})
help_button.add_class("tooltip-trigger")

# Texto de "¿Qué es GPT-2?"
help_text = widgets.HTML("<span style='color: #ced4da; font-family: sans-serif;'>¿Quieres saber más del chatbot? Click en </span>")

# Icono a la derecha
help_box = widgets.HBox([help_text, help_button], layout={'align_items': 'center', 'justify_content': 'center', 'margin_top': '15px'})


# Pantalla principal donde se muestra el historial del chat.
# Es un widget HTML vacío al principio, que se actualiza a medida que se ingresa prompts
chat_display = widgets.HTML(value=f"<div id='chat_container_ipy'></div>")

# --- Texto guia ---
instruction_html = widgets.HTML(
    value='<p style="text-align: center; color: #ced4da; font-family: sans-serif;">Si tienes alguna duda sobre el personal, contacto o colaboracion con nuestro Laboratorio, da click:</p>'
)

# Botones de sugerencia de prompts.
# Primero, definimos los textos que queremos mostrar. Se pueden poner mas pero se opto por 3
prompt_texts = [
    "¿Quién es el director del laboratorio Alan Turing?",
    "¿Cuál es la ubicación del laboratorio Alan Turing?",
    "¿Cuales son los integrantes del laboratorio Alan Turing?",
    "¿Que proyectos se hacen dentro laboratorio Alan Turing?",
]
# Usamos una "list comprehension" y por cada texto en 'prompt_texts', creamos un widget de boton. Asi se pueden poner mas prompts predefinidos
prompt_buttons = [widgets.Button(description=text, layout={'flex': '1'}) for text in prompt_texts]

# Organizamos los botones en una HBox (caja horizontal) para que aparezcan uno al lado del otro.
prompt_box = widgets.HBox(prompt_buttons, layout=widgets.Layout(justify_content='center', margin='10px 0'))
prompt_box.add_class("prompt-buttons") # Le asignamos una clase CSS para aplicar el css custom que se hizo


# Cuadro de texto del usuario y botones de acción (Enviar/Limpiar).
text_input = widgets.Text(placeholder="Si tienes otra consulta escribe aqui...", layout={'width': '70%'})
send_button = widgets.Button(description="Enviar", icon='paper-plane')
send_button.add_class("send-button")
clear_button = widgets.Button(description="Limpiar Chat", icon='trash')
clear_button.add_class("clear-button")

# Organizamos estos tres widgets en otra HBox.
input_box = widgets.HBox([text_input, send_button, clear_button], layout=widgets.Layout(width='100%', justify_content='center'))

footer_html = widgets.HTML(f"""
    <div id="footer_ipy">
        <a href="https://laboratorio-ia.epn.edu.ec/es/" target="_blank" class="lab-link">
            Laboratorio de Investigación en Inteligencia y Visión Artificial “Alan Turing”
        </a>
        <div class="social-links">
            <a href="https://www.facebook.com/laboratorio.IA.EPN" target="_blank">Facebook</a> |
            <a href="https://www.linkedin.com/company/laboratorio-ia-alan-turing/" target="_blank">LinkedIn</a> |
            <a href="https://www.youtube.com/channel/UCWE-OhGIh4rB6bp_MwKWQwQ" target="_blank">YouTube</a> |
            <a href="https://www.instagram.com/labturing.ia.epn/?utm_source=qr#" target="_blank">Instagram</a>
        </div>
    </div>
""")
# --- FUNCIONES DE LA INTERFAZ ---
# Estas funciones definen que sucede
# cuando el usuario interactúa con un widget.

def update_chat_display():

    # Función auxiliar que toma el historial guardado en la lista 'chat_history_html'
    # y lo muestra en el widget 'chat_display'.

    chat_display.value = f"<div id='chat_container_ipy'>{''.join(chat_history_html)}</div>"

    # 2. Inyecta JavaScript para forzar el desplazamiento al final
    # Esta línea encuentra el contenedor y ajusta su scroll al valor máximo (el final)
    scroll_js = """
    <script>
        var objDiv = document.getElementById("chat_container_ipy");
        if (objDiv) {
            objDiv.scrollTop = objDiv.scrollHeight;
        }
    </script>
    """
    # Muestra el script JS
    display(HTML(scroll_js))

def handle_send(message_text):

    #Función que se ejecuta cada vez que el usuario envía un mensaje.

    if not message_text: return # No hacer nada si el mensaje está vacío.
    text_input.value = ""
    # Añadir el mensaje del usuario al historial y mostrarlo inmediatamente.
    user_html = f"<div class='message-row'><div class='message-bubble user-bubble'>{message_text}</div></div>"
    chat_history_html.append(user_html)

    # Animacion de 3 puntos
    thinking_html = f"""
    <div class='message-row'>
        <div class='avatar-container'><img src='data:image/png;base64,{avatar_b64}'></div>
        <div class='message-bubble bot-bubble thinking-message'>
            <div class="typing-indicator">
                <span></span>
                <span></span>
                <span></span>
            </div>
        </div>
    </div>"""
    chat_history_html.append(thinking_html)
    update_chat_display()

    # Importante: Llamar al modelo de IA para obtener la respuesta.
    response_text = flan_predict(message_text)

    # Quitar el mensaje de animacion y añadir la respuesta final.
    chat_history_html.pop() # Se elimina la animacion
    bot_html = f"<div class='message-row'><div class='avatar-container'><img src='data:image/png;base64,{avatar_b64}'></div><div class='message-bubble bot-bubble'>{response_text}</div></div>"
    chat_history_html.append(bot_html)
    update_chat_display()
    # text_input.value = "" # Se limpia el cuadro de texto para el siguiente mensaje.

# Esta función se activa cuando se hace clic en un botón de prompts pre hechos
def on_prompt_click(b):

    # y se pasa a la función principal.
    handle_send(b.description)

#Esta función se activa con el botón 'Limpiar', reinicia el historial y actualiza la pantalla.
def on_clear_click(b):

    global chat_history_html # 'global' nos permite modificar la variable que fuera de la función.
    chat_history_html = []
    update_chat_display()

#Esta funcion controla la aparicion para mas informacion
def show_info_panel(b):
    info_panel.layout.visibility = 'visible'
    info_panel.layout.opacity = '1'
    info_panel.layout.max_height = '500px'
    info_panel.layout.padding = '20px'

def hide_info_panel(b):
    info_panel.layout.visibility = 'hidden'
    info_panel.layout.opacity = '0'
    info_panel.layout.max_height = '0px'
    info_panel.layout.padding = '0'

# --- CONEXIÓN DE EVENTOS ---
# Se conecta cada acción del usuario (click o Enter)
# con una de las funciones que definimos arriba.

# Se usa 'lambda' para crear una pequeña función anónima que captura el valor del widget en ese momento.
send_button.on_click(lambda b: handle_send(text_input.value))
text_input.on_submit(lambda sender: handle_send(sender.value)) # 'on_submit' es decir con Enter.
clear_button.on_click(on_clear_click)
# Se itera sobre cada botón de prompt pre hecho y se le asigna la misma función de clic.
for btn in prompt_buttons:
    btn.on_click(on_prompt_click)

# --- Conexiones para el panel de mas informacion ---
help_button.on_click(show_info_panel)
close_button.on_click(hide_info_panel)

# Aquí se juntan todos los widgets en el orden deseado.
# Usamos una VBox (Caja vertical) para poner los elementos uno abajo de otro
final_interface = widgets.VBox([
    header_html,
    help_box,
    info_panel,
    instruction_html,
    prompt_box,
    chat_display,
    input_box,
    footer_html
])
# se aplica el estilo principal a todo el contenedor.
final_interface.add_class("main_container_ipy")

# muestrar todo el widget unido en jupyter.
display(final_interface)

C:\Users\NW\AppData\Local\Temp\ipykernel_5320\3355439612.py:215: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(lambda sender: handle_send(sender.value)) # 'on_submit' es decir con Enter.
